In [53]:
import numpy as np
from collections import deque
import heapq

In [54]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [55]:
def print_tree(node, level=0, prefix="Root: "):
    if node is not None:
        print(" " * (4 * level) + prefix + str(node.val))
        print_tree(node.left, level + 1, "L--- ")
        print_tree(node.right, level + 1, "R--- ")

## Проверка корректности кучи

Напишите функцию, которая проверяет, является ли заданный массив корректной кучей (минимум или максимум). Алгоритм должен проверить, что все узлы удовлетворяют свойству кучи.

Если у нас есть массив `arr`, то для любого индекса `i`:

- Левый ребёнок находится по индексу `2*i + 1`
- Правый ребёнок — по индексу `2*i + 2`
- Родитель (если нужен) — по индексу `(i - 1) // 2`

- Левый ребёнок узла `i` находится по индексу `2*i + 1`
- Чтобы левый ребёнок ещё был в пределах массива, должно быть:

```
2*i + 1 < n
→ 2*i < n - 1
→ i < (n - 1) / 2
```

In [56]:
def is_max_min_heap(arr, check=max):
    n = len(arr)
    if n == 0:
        return True

    for i in range(n // 2):
        left_child = 2 * i + 1
        right_child = 2 * i + 2

        if left_child < n and check(arr[i], arr[left_child]) != arr[i]: # check -> min -> min(arr[i], arr[child] -> for True equals arr[i])
            return False
        if right_child < n and check(arr[i], arr[right_child]) != arr[i]:
            return False

    return True

In [57]:
print(is_max_min_heap([9, 5, 6, 2, 3], check=max))
print(is_max_min_heap([1, 3, 2, 7, 6], check=min))
print(is_max_min_heap([1, 6, 2, 7, 3], check=min))

True
True
False


## Проверка корректности с обходом в ширину



In [58]:
def is_max_heap(root):
    if not root:
        return True

    queue = deque([root])
    should_be_leaf = False # Flag

    while queue:
        current = queue.popleft() # текущий узел дерева, проверим его потомков, добавим их в очередь и т.д.

        if current.left:
            if should_be_leaf or current.left.val > current.val:
                return False
            queue.append(current.left)
        else:
            should_be_leaf = True  # слева ничего —> дальше листья

        if current.right:
            if should_be_leaf or current.right.val > current.val:
                return False
            queue.append(current.right)
        else:
            should_be_leaf = True  # справа ничего —> листья

    return True

In [59]:
root = TreeNode(10)
root.left = TreeNode(9)
root.right = TreeNode(8)
root.left.left = TreeNode(4)
root.left.right = TreeNode(7)

print_tree(root)
print('-' * 20)
print(is_max_heap(root))

Root: 10
    L--- 9
        L--- 4
        R--- 7
    R--- 8
--------------------
True


## Полное бинарное дерево

```
В обходе в ширину (BFS):
мы посещаем: 8 → 3 → 9 → 11 → 6

Что происходит с очередью:
8: есть левый (3) и правый (9) → всё ок, добавляем обоих в очередь
3: есть левый (11) и правый (6) → тоже всё ок, добавляем обоих
9: нет детей → теперь мы устанавливаем seen_none = True
Это нормально — мы дошли до «зоны листьев»
11: нет детей → всё ещё ок, потому что seen_none уже True
6: нет детей → тоже всё хорошо
```

In [60]:
def is_complete_binary_tree(root):
    if not root:
        return True

    queue = deque([root])
    seen_none = False # Flag

    while queue:
        current = queue.popleft()
        if current is None:
            seen_none = True # hit a gap in the tree
        else:
            if seen_none:
                return False  # found node after a gap → not complete
            queue.append(current.left)
            queue.append(current.right)

    return True

In [61]:
root1 = TreeNode(1)
root1.left = TreeNode(2)
root1.right = TreeNode(3)
root1.left.left = TreeNode(4)
root1.left.right = TreeNode(5)
root1.right.left = TreeNode(6)

print_tree(root1)
print(is_complete_binary_tree(root1))
print('-' * 20)

root2 = TreeNode(1)
root2.left = TreeNode(2)
root2.right = TreeNode(3)
root2.left.left = TreeNode(4)
root2.right.right = TreeNode(6)

print_tree(root2)
print(is_complete_binary_tree(root2))

Root: 1
    L--- 2
        L--- 4
        R--- 5
    R--- 3
        L--- 6
True
--------------------
Root: 1
    L--- 2
        L--- 4
    R--- 3
        R--- 6
False


## Объединение `K` отсортированных массивов
Реализуйте функцию, которая объединяет `K` отсортированных массивов в один отсортированный массив. Используйте мин-кучу для хранения наименьших элементов текущих массивов, что позволит извлекать их по очереди, сохраняя порядок.

Сложность
1. Инициализация кучи:
Мы проходим по каждому из K массивов и добавляем первый элемент каждого массива в мин-кучу.
Это занимает `O(K)` времени, так как мы добавляем только K элементов.

2. Извлечение элементов из кучи:
В каждом цикле `while` мы извлекаем наименьший элемент из кучи.
Извлечение элемента из мин-кучи занимает `O(log K)` времени, так как в худшем случае нам нужно будет перестраивать кучу.
В процессе извлечения мы также добавляем следующий элемент из того же массива в кучу.
Это также занимает `O(log K)` времени.

3. Общее количество извлечений:
В худшем случае, если все массивы имеют в сумме N элементов,
мы будем извлекать элементы `N` раз.
Поэтому общее время, затраченное на извлечение и добавление элементов, составит `O(N log K)`.

Итоговая временная сложность:
`O(K + N log K)`, где

`K` — количество массивов,

`N` — общее количество элементов во всех массивах.

In [62]:
def merge_k_sorted_arrays(arrays):
    heap = []
    result = []
    print("Инициализация кучи:")
    for i, arr in enumerate(arrays):
        if arr:
            heapq.heappush(heap, (arr[0], i, 0)) # (значение, индекс массива, индекс внутри массива)
            print(f"Добавлен элемент {arr[0]} из массива {i}: {heap}")
    print("\n Объединение:")
    while heap:
        val, arr_idx, elem_idx = heapq.heappop(heap)
        result.append(val)
        print(f"Взяли минимальный элемент: {val} (из массива {arr_idx}, индекс {elem_idx})")
        print(f"-> Текущий результат: {result}")

        if elem_idx + 1 < len(arrays[arr_idx]):
            next_val = arrays[arr_idx][elem_idx + 1]
            heapq.heappush(heap, (next_val, arr_idx, elem_idx + 1))
            print(f"Добавлен следующий элемент: {next_val} (из массива {arr_idx}, индекс {elem_idx + 1})")
            print(f"-> Текущая куча: {heap}")
        else:
            print(f"Массив {arr_idx} закончился")
    return result

In [63]:
arrays = [
    [1, 3, 5, 7],
    [2, 4, 6],
    [0, 8, 9, 11]
]

merge_k_sorted_arrays(arrays)

Инициализация кучи:
Добавлен элемент 1 из массива 0: [(1, 0, 0)]
Добавлен элемент 2 из массива 1: [(1, 0, 0), (2, 1, 0)]
Добавлен элемент 0 из массива 2: [(0, 2, 0), (2, 1, 0), (1, 0, 0)]

 Объединение:
Взяли минимальный элемент: 0 (из массива 2, индекс 0)
-> Текущий результат: [0]
Добавлен следующий элемент: 8 (из массива 2, индекс 1)
-> Текущая куча: [(1, 0, 0), (2, 1, 0), (8, 2, 1)]
Взяли минимальный элемент: 1 (из массива 0, индекс 0)
-> Текущий результат: [0, 1]
Добавлен следующий элемент: 3 (из массива 0, индекс 1)
-> Текущая куча: [(2, 1, 0), (8, 2, 1), (3, 0, 1)]
Взяли минимальный элемент: 2 (из массива 1, индекс 0)
-> Текущий результат: [0, 1, 2]
Добавлен следующий элемент: 4 (из массива 1, индекс 1)
-> Текущая куча: [(3, 0, 1), (8, 2, 1), (4, 1, 1)]
Взяли минимальный элемент: 3 (из массива 0, индекс 1)
-> Текущий результат: [0, 1, 2, 3]
Добавлен следующий элемент: 5 (из массива 0, индекс 2)
-> Текущая куча: [(4, 1, 1), (8, 2, 1), (5, 0, 2)]
Взяли минимальный элемент: 4 (из ма

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11]

## K-ый наименьший эелемент в BST

- Создаём стек для хранения узлов и счётчик
- Начинаем обход с корня
- Идём влево до конца
- Записываем значения в стек
- Извлекаем значение из вершины стека
- Увеличиваем счётчик
- Если `counter == k`, возвращаем значение узла
- Если нет — переходим в правое поддерево
- Переписываем текущее значение на значение из правого поддерева

In [64]:
def kth_smallest(root, k):
    stack = []
    counter = 0
    current = root

    print("Начинаем in-order обход BST...\n")

    while stack or current:
        while current: # идём влево до самого конца
            print(f"Идём влево: добавляем {current.val} в стек")
            stack.append(current)
            current = current.left
        current = stack.pop()
        counter += 1
        print(f"\nИзвлекли из стека: {current.val}, это {counter}-й элемент")

        if counter == k:
            print(f"\nНайден k-й наименьший элемент: {current.val}")
            return current.val
        print(f"Переход в правое поддерево от {current.val}")
        current = current.right
    return None # если k больше количества узлов

In [65]:
root = TreeNode(16)
root.left = TreeNode(10)
root.right = TreeNode(22)
root.left.left = TreeNode(6)
root.left.right = TreeNode(12)
root.right.left = TreeNode(18)
root.right.right = TreeNode(24)

root.left.left.left = TreeNode(2)
root.left.left.right = TreeNode(8)
root.left.right.left = TreeNode(11)
root.left.right.right = TreeNode(13)

root.right.left.left = TreeNode(17)
root.right.left.right = TreeNode(21)
root.right.right.left = TreeNode(23)
root.right.right.right = TreeNode(27)

print_tree(root)
print('-' * 20)
kth_smallest(root, 5)

Root: 16
    L--- 10
        L--- 6
            L--- 2
            R--- 8
        R--- 12
            L--- 11
            R--- 13
    R--- 22
        L--- 18
            L--- 17
            R--- 21
        R--- 24
            L--- 23
            R--- 27
--------------------
Начинаем in-order обход BST...

Идём влево: добавляем 16 в стек
Идём влево: добавляем 10 в стек
Идём влево: добавляем 6 в стек
Идём влево: добавляем 2 в стек

Извлекли из стека: 2, это 1-й элемент
Переход в правое поддерево от 2

Извлекли из стека: 6, это 2-й элемент
Переход в правое поддерево от 6
Идём влево: добавляем 8 в стек

Извлекли из стека: 8, это 3-й элемент
Переход в правое поддерево от 8

Извлекли из стека: 10, это 4-й элемент
Переход в правое поддерево от 10
Идём влево: добавляем 12 в стек
Идём влево: добавляем 11 в стек

Извлекли из стека: 11, это 5-й элемент

Найден k-й наименьший элемент: 11


11

## Balance Factor

In [66]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.balance_factor = 0 # добавили баланс фактор в узел

In [67]:
def calculate_heights_and_balance(root):
    if not root:
        return 0

    left_height = calculate_heights_and_balance(root.left)  # рекурсивный вызов для левого и правого поддеревьев
    right_height = calculate_heights_and_balance(root.right)

    root.balance_factor = left_height - right_height
    print(f"Узел {root.val}: высота слева = {left_height}, справа = {right_height}, баланс = {root.balance_factor}")

    return 1 + max(left_height, right_height)

In [69]:
root = TreeNode(10)
root.left = TreeNode(5)
root.right = TreeNode(15)
root.left.left = TreeNode(2)
root.left.right = TreeNode(7)

print_tree(root)
print('-' * 20)

calculate_heights_and_balance(root)

Root: 10
    L--- 5
        L--- 2
        R--- 7
    R--- 15
--------------------
Узел 2: высота слева = 0, справа = 0, баланс = 0
Узел 7: высота слева = 0, справа = 0, баланс = 0
Узел 5: высота слева = 1, справа = 1, баланс = 0
Узел 15: высота слева = 0, справа = 0, баланс = 0
Узел 10: высота слева = 2, справа = 1, баланс = 1


3

## Преобразование в зеркальное дерево
Нужно реализовать алгоритм, который перевернёт бинарное дерево "вверх ногами", т.е. поменяет местами левые и правые поддеревья каждого узла.

In [70]:
def mirror_tree_iterative(root):
    if not root:
        return None

    queue = deque([root])
    while queue:
        current = queue.popleft()
        current.left, current.right = current.right, current.left
        if current.left:
            queue.append(current.left)
        if current.right:
            queue.append(current.right)
    return root

In [71]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)
root.right.right = TreeNode(6)

print_tree(root)
print('-' * 20)
mirror_tree_iterative(root)
print("\nОтзеркаленое дерево:")
print_tree(root)


Root: 1
    L--- 2
        L--- 4
        R--- 5
    R--- 3
        R--- 6
--------------------

Отзеркаленое дерево:
Root: 1
    L--- 3
        L--- 6
    R--- 2
        L--- 5
        R--- 4
